## Collection des images

In [1]:
# !pip install opencv-python

## Importation des Packages

In [2]:
# OpenCV
import cv2

# uuid
import uuid

## Opérations sur le Système d'exploitation
import os

# la gestion du temps
import time

## Définition des images à collecter

In [ ]:
# labels = ['thumbsup', 'thumbsdown', 'thankyou', 'livelong']
labels = ['lunette', 'brosseadent', 'patedentifrice', 'montre', 'peignecheveux',  'telephone', 'livre',]
number_images = 7

## Configuration du dossier des images

In [ ]:
IMAGES_PATH = os.path.join('.', 'tensorflow', 'workspace', 'images', 'collectedimages')

In [ ]:
IMAGES_PATH

In [ ]:
if not os.path.exists(IMAGES_PATH):
    if os.name == 'posix':
        !mkdir - {IMAGES_PATH}
    if os.name == 'nt':
        !mkdir {IMAGES_PATH}
        
for label in labels:
    path = os.path.join(IMAGES_PATH, label)
    if not os.path.exists(path):
        !mkdir {path}

## Capture des Images WebCam

In [ ]:
for label in labels:
    cap = cv2.VideoCapture(0) # Se connecter à son webcam
    print(f'Récuperer l\'image pour {label}')
    time.sleep(5)
    for imgnum in range(number_images):
        print('Collecting image {}'.format(imgnum))
        ret, frame = cap.read()
        imgname = os.path.join(IMAGES_PATH,label,label+'.'+'{}.jpg'.format(str(uuid.uuid1())))
        cv2.imwrite(imgname, frame)
        cv2.imshow('frame', frame)
        time.sleep(2)

        if cv2.waitKey(1) & 0xFF == ord('q'):
            break
cap.release()
cv2.destroyAllWindows()

## Etiquetage des Images

In [ ]:
# !pip install --upgrade pyqt5 lxml

In [ ]:
# !pip list

In [ ]:
LABEL_IMAGES_PATH = os.path.join('.', 'tensorflow', 'labelimg')

In [ ]:
if not os.path.exists(LABEL_IMAGES_PATH):
    !mkdir {LABEL_IMAGES_PATH}
    !git clone https://github.com/tzutalin/labelImg {LABEL_IMAGES_PATH}

In [ ]:
if os.name == 'posix':
    !make qt5py3
if os.name =='nt':
    !cd {LABEL_IMAGES_PATH} && pyrcc5 -o libs/resources.py resources.qrc

In [ ]:
!cd {LABEL_IMAGES_PATH} && python labelImg.py

## Déplacez-les dans une partition de train et de test

## FACULTATIF -  Compressez-les pour l'entrinement Colab¶

In [ ]:
TRAIN_PATH = os.path.join('Tensorflow', 'workspace', 'images', 'train')
TEST_PATH = os.path.join('Tensorflow', 'workspace', 'images', 'test')
ARCHIVE_PATH = os.path.join('Tensorflow', 'workspace', 'images', 'archive.tar.gz')
ARCHIVE_PATH_Win = os.path.join('Tensorflow', 'workspace', 'images', 'archive.tar.gz')

In [ ]:
!tar -czf {ARCHIVE_PATH} {TRAIN_PATH} {TEST_PATH}

In [ ]:
# !tar -czf {ARCHIVE_PATH_Win} {TRAIN_PATH} {TEST_PATH}

## Entraînez le modèle tfod à l'aide des scripts fournis, cela peut prendre un peu de temps en fonction de votre machine

In [3]:
CUSTOM_MODEL_NAME = 'my_ssd_mobnet' 
PRETRAINED_MODEL_NAME = 'ssd_mobilenet_v2_fpnlite_320x320_coco17_tpu-8'
PRETRAINED_MODEL_URL = 'http://download.tensorflow.org/models/object_detection/tf2/20200711/ssd_mobilenet_v2_fpnlite_320x320_coco17_tpu-8.tar.gz'
TF_RECORD_SCRIPT_NAME = 'generate_tfrecord.py'
LABEL_MAP_NAME = 'label_map.pbtxt'

In [4]:
paths = {
    'WORKSPACE_PATH': os.path.join('Tensorflow', 'workspace'),
    'SCRIPTS_PATH': os.path.join('Tensorflow','scripts'),
    'APIMODEL_PATH': os.path.join('Tensorflow','models'),
    'ANNOTATION_PATH': os.path.join('Tensorflow', 'workspace','annotations'),
    'IMAGE_PATH': os.path.join('Tensorflow', 'workspace','images'),
    'MODEL_PATH': os.path.join('Tensorflow', 'workspace','models'),
    'PRETRAINED_MODEL_PATH': os.path.join('Tensorflow', 'workspace','pre-trained-models'),
    'CHECKPOINT_PATH': os.path.join('Tensorflow', 'workspace','models',CUSTOM_MODEL_NAME), 
    'OUTPUT_PATH': os.path.join('Tensorflow', 'workspace','models',CUSTOM_MODEL_NAME, 'export'), 
    'TFJS_PATH':os.path.join('Tensorflow', 'workspace','models',CUSTOM_MODEL_NAME, 'tfjsexport'), 
    'TFLITE_PATH':os.path.join('Tensorflow', 'workspace','models',CUSTOM_MODEL_NAME, 'tfliteexport'), 
    'PROTOC_PATH':os.path.join('Tensorflow','protoc')
 }

In [5]:
# files = {
#     'PIPELINE_CONFIG':os.path.join('Tensorflow', 'workspace','models', CUSTOM_MODEL_NAME, 'pipeline.config'),
#     'TF_RECORD_SCRIPT': os.path.join(paths['SCRIPTS_PATH'], TF_RECORD_SCRIPT_NAME), 
#     'LABELMAP': os.path.join(paths['ANNOTATION_PATH'], LABEL_MAP_NAME)
# }

In [6]:
# for path in paths.values():
#     if not os.path.exists(path):
#         if os.name == 'posix':
#             !mkdir -p {path}
#         if os.name == 'nt':
#             !mkdir {path}

##  Téléchargez des modèles pré-entraînés de modèles TF à partir de Tensorflow Model Zoo et installez TFOD

In [7]:
if os.name=='nt':
    !pip install wget
    import wget

In [ ]:
# if not os.path.exists(os.path.join(paths['APIMODEL_PATH'], 'research', 'object_detection')):
#     !git clone https://github.com/tensorflow/models {paths['APIMODEL_PATH']}

In [ ]:
# if os.name=='posix':  
#     !apt-get install protobuf-compiler
#     !cd Tensorflow/models/research && protoc object_detection/protos/*.proto --python_out=. && cp object_detection/packages/tf2/setup.py . && python -m pip install . 
    
# if os.name=='nt':
#     url="https://github.com/protocolbuffers/protobuf/releases/download/v3.15.6/protoc-3.15.6-win64.zip"
#     wget.download(url)
#     !move protoc-3.15.6-win64.zip {paths['PROTOC_PATH']}
#     !cd {paths['PROTOC_PATH']} && tar -xf protoc-3.15.6-win64.zip
#     os.environ['PATH'] += os.pathsep + os.path.abspath(os.path.join(paths['PROTOC_PATH'], 'bin'))   
#     !cd Tensorflow/models/research && protoc object_detection/protos/*.proto --python_out=. && copy object_detection\\packages\\tf2\\setup.py setup.py && python setup.py build && python setup.py install
#     !cd Tensorflow/models/research/slim && pip install -e . 

In [ ]:
!pip install tensorflow --upgrade

In [ ]:
!pip uninstall protobuf matplotlib -y
!pip install protobuf matplotlib==3.2

In [8]:
import object_detection

In [9]:
!pip list

Package                      Version             Editable project location
---------------------------- ------------------- -------------------------------------------------------------------------------------
absl-py                      1.0.0
apache-beam                  2.37.0rc2
asttokens                    2.0.5
astunparse                   1.6.3
avro-python3                 1.10.2
backcall                     0.2.0
black                        22.1.0
cachetools                   5.0.0
certifi                      2021.10.8
charset-normalizer           2.0.12
click                        8.0.4
colorama                     0.4.4
contextlib2                  21.6.0
cycler                       0.11.0
Cython                       3.0.0a10
debugpy                      1.5.1
decorator                    5.1.1
entrypoints                  0.4
executing                    0.8.2
flatbuffers                  2.0
gast                         0.5.3
google-auth                  2.6.0
google-a

In [10]:
if os.name =='posix':
    !wget {PRETRAINED_MODEL_URL}
    !mv {PRETRAINED_MODEL_NAME+'.tar.gz'} {paths['PRETRAINED_MODEL_PATH']}
    !cd {paths['PRETRAINED_MODEL_PATH']} && tar -zxvf {PRETRAINED_MODEL_NAME+'.tar.gz'}
if os.name == 'nt':
    wget.download(PRETRAINED_MODEL_URL)
    !move {PRETRAINED_MODEL_NAME+'.tar.gz'} {paths['PRETRAINED_MODEL_PATH']}
    !cd {paths['PRETRAINED_MODEL_PATH']} && tar -zxvf {PRETRAINED_MODEL_NAME+'.tar.gz'}

100% [.................................................................] 20515344 / 20515344        1 fichier(s) d‚plac‚(s).


x ssd_mobilenet_v2_fpnlite_320x320_coco17_tpu-8/
x ssd_mobilenet_v2_fpnlite_320x320_coco17_tpu-8/checkpoint/
x ssd_mobilenet_v2_fpnlite_320x320_coco17_tpu-8/checkpoint/ckpt-0.data-00000-of-00001
x ssd_mobilenet_v2_fpnlite_320x320_coco17_tpu-8/checkpoint/checkpoint
x ssd_mobilenet_v2_fpnlite_320x320_coco17_tpu-8/checkpoint/ckpt-0.index
x ssd_mobilenet_v2_fpnlite_320x320_coco17_tpu-8/pipeline.config
x ssd_mobilenet_v2_fpnlite_320x320_coco17_tpu-8/saved_model/
x ssd_mobilenet_v2_fpnlite_320x320_coco17_tpu-8/saved_model/saved_model.pb
x ssd_mobilenet_v2_fpnlite_320x320_coco17_tpu-8/saved_model/variables/
x ssd_mobilenet_v2_fpnlite_320x320_coco17_tpu-8/saved_model/variables/variables.data-00000-of-00001
x ssd_mobilenet_v2_fpnlite_320x320_coco17_tpu-8/saved_model/variables/variables.index


## Create Label Map

In [ ]:
labels = [{'name':'ThumbsUp', 'id':1}, {'name':'ThumbsDown', 'id':2}, {'name':'ThankYou', 'id':3}, {'name':'LiveLong', 'id':4}]
labels = [
    {'name': 'lunette', 'id': 1}, 
    {'name': 'brosseadent', 'id': 2},
    {'name': 'patedentifrice', 'id': 3},
    {'name': 'montre', 'id': 4},
    {'name': 'peignecheveux', 'id': 5},
    {'name': 'telephone', 'id': 6},
    {'name': 'livre', 'id': 7}
]
with open(files['LABELMAP'], 'w') as f:
    for label in labels:
        f.write('item { \n')
        f.write('\tname:\'{}\'\n'.format(label['name']))
        f.write('\tid:{}\n'.format(label['id']))
        f.write('}\n')